In [1]:
import sys
sys.path.append("../")
import pandas as pd
import datetime as dt
from api.oanda_api import OandaApi
from dateutil import parser
import timeit
import plotly.graph_objects as go
from technicals import trend_checker
from technicals import zone_detector
pd.set_option('display.max_columns', None)

In [2]:
df_an = pd.read_pickle('../data/EUR_USD_H4.pkl').copy()
df_an['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df_an.time]
trend_checker.apply_downtrend(df_an)
df_an.head(20)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,sTime,ma_10,ma_50,in_downtrend
49,2024-10-11 01:00:00+00:00,9451,1.09374,1.09408,1.09296,1.09356,1.09367,1.09400,1.09289,1.09349,1.09382,1.09415,1.09303,1.09363,s24-10-11 01:00,1.093568,1.100100,True
50,2024-10-11 05:00:00+00:00,16781,1.09358,1.09538,1.09322,1.09496,1.09350,1.09530,1.09314,1.09489,1.09365,1.09546,1.09330,1.09503,s24-10-11 05:00,1.093648,1.099735,True
51,2024-10-11 09:00:00+00:00,24833,1.09494,1.09512,1.09261,1.09332,1.09487,1.09504,1.09253,1.09324,1.09501,1.09519,1.09269,1.09340,s24-10-11 09:00,1.093580,1.099320,True
52,2024-10-11 13:00:00+00:00,31036,1.09331,1.09525,1.09281,1.09449,1.09324,1.09518,1.09273,1.09441,1.09338,1.09532,1.09288,1.09457,s24-10-11 13:00,1.093623,1.099004,True
53,2024-10-11 17:00:00+00:00,11351,1.09448,1.09475,1.09325,1.09370,1.09440,1.09467,1.09317,1.09360,1.09457,1.09483,1.09333,1.09379,s24-10-11 17:00,1.093582,1.098736,True
54,2024-10-13 21:00:00+00:00,7016,1.09328,1.09338,1.09154,1.09216,1.09319,1.09329,1.09147,1.09208,1.09336,1.09352,1.09161,1.09223,s24-10-13 21:00,1.093468,1.098470,True
55,2024-10-14 01:00:00+00:00,11313,1.09215,1.09293,1.09180,1.09238,1.09208,1.09285,1.09171,1.09229,1.09222,1.09301,1.09188,1.09247,s24-10-14 01:00,1.093264,1.098182,True
56,2024-10-14 05:00:00+00:00,17397,1.09237,1.09318,1.09224,1.09270,1.09229,1.09310,1.09216,1.09262,1.09245,1.09326,1.09232,1.09278,s24-10-14 05:00,1.093472,1.097905,True
57,2024-10-14 09:00:00+00:00,19748,1.09270,1.09367,1.09074,1.09094,1.09263,1.09359,1.09065,1.09087,1.09278,1.09375,1.09081,1.09102,s24-10-14 09:00,1.093196,1.097598,True
58,2024-10-14 13:00:00+00:00,25470,1.09096,1.09276,1.09008,1.09016,1.09088,1.09267,1.09000,1.09008,1.09104,1.09284,1.09016,1.09023,s24-10-14 13:00,1.092837,1.097256,True


In [3]:
support_levels, resistance_levels = zone_detector.find_support_resistance(df_an)

In [4]:
fig = trend_checker.highlight_downtrend_candles(df_an)
x_start = df_an['sTime'].iloc[0]
x_end = df_an['sTime'].iloc[-1]

# Plot support levels
for level in support_levels:
    fig.add_trace(go.Scatter(
        x=[x_start, x_end],
        y=[level, level],
        mode='lines',
        name=f'Support {level}',
        line=dict(color='green', width=1)
    ))

    # Plot resistance levels
for level in resistance_levels:
    fig.add_trace(go.Scatter(
        x=[x_start, x_end],
        y=[level, level],
        mode='lines',
        name=f'Resistance {level}',
        line=dict(color='red', width=1)
    ))

fig.show()